## Parse data files from PushShift.io for selected subreddits using pyWren

In [1]:
# Importing the necessary packages 
import json
import bz2
import pywren

In [2]:
# Creating appropriate pywren buckets
! pywren create_config 
! pywren create_role
! pywren create_bucket
! pywren deploy_lambda

Traceback (most recent call last):
  File "/Users/angelicabosko/anaconda3/bin/pywren", line 8, in <module>
    sys.exit(main())
  File "/Users/angelicabosko/anaconda3/lib/python3.7/site-packages/pywren/scripts/pywrencli.py", line 711, in main
    return cli() # pylint: disable=no-value-for-parameter
  File "/Users/angelicabosko/anaconda3/lib/python3.7/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/Users/angelicabosko/anaconda3/lib/python3.7/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/Users/angelicabosko/anaconda3/lib/python3.7/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/Users/angelicabosko/anaconda3/lib/python3.7/site-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/Users/angelicabosko/anaconda3/lib/python3.7/site-packages/click/core.py", line 610, in invoke
    ret

In [2]:
# Testing some sample data
data = []
for line in open('reddit_sample_data.json', 'r'):
    data.append(json.loads(line))

FileNotFoundError: [Errno 2] No such file or directory: 'reddit_sample_data.json'

In [9]:
print(data[0])

{'author': 'Dethcola', 'author_flair_css_class': '', 'author_flair_text': 'Clairemont', 'body': 'A quarry', 'can_gild': True, 'controversiality': 0, 'created_utc': 1506816000, 'distinguished': None, 'edited': False, 'gilded': 0, 'id': 'dnqik14', 'is_submitter': False, 'link_id': 't3_73ieyz', 'parent_id': 't3_73ieyz', 'permalink': '/r/sandiego/comments/73ieyz/best_place_for_granite_counter_tops/dnqik14/', 'retrieved_on': 1509189606, 'score': 3, 'stickied': False, 'subreddit': 'sandiego', 'subreddit_id': 't5_2qq2q'}


In [3]:
# Subreddits we are pulling data from
target_subreddits = ['TheRedPill']

In [15]:
# comment parser for single file (month)
# Code adapted from: https://github.com/AhmedSoli/Reddit-Politics/blob/master/01_Content_Analysis/PreProcessing/.ipynb_checkpoints/010_ExtractCommentsTextCorpus-checkpoint.ipynb
from IPython.display import display, clear_output
import json
import os
import multiprocessing
import pickle
import bz2
import lzma

def extract_comments(file_path):
    '''
    '''
    month_comments = {}
    for sub in target_subreddits:
        month_comments[sub] = []
    if '.bz2' in file_path:
        o = bz2.open

    with o(file_path,'rt') as comments_info:
            for i,comment in enumerate(comments_info):
                # load comment into json object
                comment = json.loads(comment)
#                 # check if comment contains information about where it was posted
#                 # all comments should include this information
#                 if 'subreddit' in comment:
#                     # check if the subreddit where the comment was posted
#                     # is in the target boards
                if comment['subreddit'] in target_subreddits:
                    # if yes append the body of the comment
                    month_comments[comment['subreddit']].append((comment['body'], comment['created_utc']))
                
                 # display progress
                if i % 1000000 == 0:
                    clear_output(wait=True)
                    print(file_path,i)
                    
    # serlialize the result for this month
    #pickle_out = open("reddit_comments/comments_corpus_TRP_" + file_path[:-4] + ".pickle","wb")
    #pickle.dump(month_comments, pickle_out)
    #pickle_out.close()
    return month_comments
    

In [16]:
# Parallelize comment scraping using pywren.
# Can either automize file search in order to store multiple file path names or just add them to iterdata list

pushshift_files = ['RC_2011-08.bz2']
pwex = pywren.default_executor()
futures = pwex.map(extract_comments, pushshift_files)

#monthly_files = pywren.get_all_results(futures) 

In [18]:
monthly_files = pywren.get_all_results(futures)

OSError: [Errno 28] No space left on device

In [13]:
extract_comments('RC_2011-08.bz2')

RC_2011-08.bz2 0


KeyboardInterrupt: 

In [103]:
TRP_2011 = pickle.load(open("reddit_comments/comments_corpus_TRPRC_2011-08.pickle","rb"))

In [104]:
TRP_2011

{'TheRedPill': []}

In [38]:
comments_corpus = {}
# Keep file pickleing for storage reasons. 
# Storing all monthly comments into one corpus for each individual subreddit
for file in sorted(push_shift):
    comments_corpus_temp = pickle.load(open("../../Results/comments_corpus_" + file[:-4] + ".pickle","rb")) # loading in monthly files without file type
    for subreddit in comments_corpus_temp:
        if subreddit not in comments_corpus: # some selected subreddits might not be present in files
            comments_corpus[subreddit] = []
        # Add comment to entire corpus of comments for a single subreddit. This might be why we want to collect utc, incase we want to run time series on the data to map key events
        comments_corpus[subreddit].extend(
            comments_corpus_temp[subreddit]
        )
        print(subreddit,len(comments_corpus[subreddit]))
                        
pickle_out = open("reddit_comments/comments_corpus.pickle","wb")
pickle.dump(comments_corpus, pickle_out)
pickle_out.close()

NameError: name 'push_shift' is not defined